In [15]:
import cv2
import numpy as np

import time
from threading import Thread

from moviepy.editor import *
from moviepy.audio.AudioClip import AudioArrayClip

BS = 16 #розмір макроблоку
Step = 4 #залежить пошукова область(2*Step-1)

VideoName = "video.flv"#відео, яке потрібно "стиснути"
Compress = "comressed_video.npy"#сюди записуємо "стиснуте" відео

audiofps = 44100#"звуків в одній секунді"
audioStart = True

#знаходить матрицю зміщення second відносно first
def findbias(first, second):
    height = first.shape[0] // BS
    width = first.shape[1] // BS
    bias = np.zeros((height, width, 2))
    t1 = Thread(target=onethread, args=(bias,0,height//4, first, second, width))
    t2 = Thread(target=onethread, args=(bias,height//4,height//2, first, second, width))
    t3 = Thread(target=onethread, args=(bias,height//2,3*height//4, first, second, width))
    t4 = Thread(target=onethread, args=(bias,3*height//4,height, first, second, width))
    t1.start()
    t2.start()
    t3.start()
    t4.start()
    t1.join()
    t2.join()
    t3.join()
    t4.join()
    return bias
def onethread(bias, fro, to, first, second, width):
    for i in range(fro, to):
        for j in range(width):
            S = Step
            imin = 0
            jmin = 0
            while(S > 0):
                im, jm = findmindif(first, second, S, i*BS, j*BS)
                imin += im
                jmin += jm
                S = S // 2
            bias[i, j, 0] = imin
            bias[i, j, 1] = jmin
            
# знаходить напрям зміщення для одного макроблоку
def findmindif(first, second, S, x, y):
    mind = 100000
    imin = 0
    jmin = 0
    block = first[x:(x+BS),y:(y+BS)]
    for i in [0, -S, S]:
        if x + i >= 0 and x + i + BS <= second.shape[0]:
            dif = mad(block, second[(x+i):(x+i+BS),y:(y+BS)])
            if mind > dif:
                #print(dif)
                mind = dif
                imin = i
                jmin = 0
    for j in [-S, S]:
        if y + j >= 0 and y + j + BS <= second.shape[0]:
            dif = mad(block, second[x:(x+BS),(y+j):(y+j+BS)])
            if mind > dif:
                mind = dif
                imin = 0
                jmin = j
    return imin, jmin

#Mean Absolute Difference
def mad(a, b):
    return 1/a.shape[0]/a.shape[1]*np.sum(np.abs(a - b))


#будує кадр з даного кадру і матриці зміщення
def buildFrame(frame, bias):
    newframe = frame.copy()
    for i in range(bias.shape[0]):
        for j in range(bias.shape[1]):
            block = frame[i*BS:(i+1)*BS,j*BS:(j+1)*BS,:]
            newframe[int(i*BS+bias[i][j][0]):int((i+1)*BS+bias[i][j][0]),int(j*BS+bias[i][j][1]):int((j+1)*BS+bias[i][j][1]),:] = block
    return newframe

#відтворює аудіо
def audio(audionp, fps):
    global audioStart
    clip = AudioArrayClip(audionp, fps)
    audioStart = True
    clip.preview()

In [91]:
#Encoder

cap = cv2.VideoCapture(VideoName)
file = open(Compress , 'wb')
difference = 0
count = 0

audioclip = AudioFileClip(VideoName)
d = audioclip.duration
au = np.zeros((int(d*audiofps), audioclip.nchannels))
for i in range(int(d*audiofps)):
    au[i] = audioclip.get_frame((i+1)/audiofps) 
np.save(file, au)#записує аудіо до файлу
    
    
start_time = time.time()
while(1):
    #зчитуємо порцію з трьох кадрів якщо це можливо
    ret, frame1 = cap.read()
    if ret == True:
        ret, frame2 = cap.read()
        if ret == True:
            np.save(file, frame2)#зберігаємо другий кадр
            first = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
            second = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
            #знаходимо матрицю зміщення першого кадру відносно другого 
            bias = findbias(second, first)
            count += 1
            newframe = buildFrame(frame2, bias)
            #рахуємо різницю між оригінальним та згенерованим кадром
            difference += mad(frame1, newframe)
            np.save(file, bias.astype(np.int8))#зберігаємо матрицю зміщення першого кадру
            ret, frame3 = cap.read()
            if ret == True:
                third = cv2.cvtColor(frame3,cv2.COLOR_BGR2GRAY)
                #знаходимо матрицю зміщення третього кадру відносно другого 
                bias = findbias(second, third)
                count += 1
                newframe = buildFrame(frame2, bias)
                difference += mad(frame3, newframe)
                np.save(file, bias.astype(np.int8))#зберігаємо матрицю зміщення третього кадру
            else:
                break
        else:
            np.save(file, frame1)#зберігаємо перший кадр, якщо він останній
            break
    else:
        break

    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
times = round(time.time() - start_time, 0)
print(f"time of encoding: {times // 60} minutes {times % 60} seconds")
print(f"mean difference between original and genereted frames: {round(difference/count, 4)}")
    
file.close()
cap.release()
cv2.destroyAllWindows()

time of encoding: 5.0 minutes 2.0 seconds
mean difference between original and genereted frames: 33.0373


In [40]:
#Decoder
file = open(Compress, 'rb')
out = cv2.VideoWriter('resultvideo1.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 15, (704,576))

audionp = np.load(file)#зчитує аудіо до відео

begin = True
start_time = time.time()
while(1):
    if(audioStart):
        try:
            frame = np.load(file)#зчитуємо матрицю з пікселями другого кадру
            try:
                bias = np.load(file)#зчитуємо матрицю зміщення першого кадру
                newframe = buildFrame(frame, bias)#відновлюємо перший кадр
                cv2.imshow('video',newframe)
                cv2.imshow('video',frame)
                out.write(newframe)
                out.write(frame)
                try:
                    bias = np.load(file)#зчитуємо матрицю зміщення третього кадру
                    newframe = buildFrame(frame, bias)#відновлюємо третій кадр
                    cv2.imshow('video',newframe)
                    out.write(newframe)
                except ValueError:
                    break
            except ValueError:
                cv2.imshow('video',frame)#якщо є маттриця з пікселями, але не має матриць зміщення, значить це останній кадр
                out.write(frame)
                break
        except ValueError:
            break
        if begin:
            #намагаюсь синхронізувати аудіо з відео обчислюючи час відтворення перших трьох кадрів
            fps = 25
            beta = 7000#можна погратись з цими параметрами
            newAudioFps = int(3/(time.time() - start_time)/fps*audiofps + beta)
            audiothread = Thread(target=audio, args = (audionp, newAudioFps))
            audioStart = False
            audiothread.start()
            begin = False
        #cv2.waitKey(0)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
times = round(time.time() - start_time, 0)
print(f"time of decoding: {times // 60} minutes {times % 60} seconds")     

file.close()
audiothread.join()
out.release()
cv2.destroyAllWindows()

#додаю аудіо до відео з результатом
videoclip = VideoFileClip("resultvideo1.avi")
normalAudio = AudioArrayClip(audionp, int(audiofps))
videoclip2 = videoclip.set_audio(normalAudio)
videoclip2.write_videofile("resultvideo1.mp4")

time of decoding: 0.0 minutes 43.0 seconds


chunk:   2%|▉                                                             | 28/1813 [00:00<00:06, 257.85it/s, now=None]

Moviepy - Building video resultvideo1.mp4.
MoviePy - Writing audio in resultvideo1TEMP_MPY_wvf_snd.mp3



t:   1%|▍                                                                   | 7/1233 [00:00<00:18, 67.59it/s, now=None]

MoviePy - Done.
Moviepy - Writing video resultvideo1.mp4




t:  87%|███████████████████████████████████████████████████████▋        | 1072/1233 [00:07<00:01, 145.28it/s, now=None]


t:   0%|                                                             | 2/3083 [43:52<1126:33:12, 1316.32s/it, now=None]

Moviepy - Done !
Moviepy - video ready resultvideo1.mp4


In [114]:
count

1234

In [7]:
shape

(576, 704, 3)